In [ ]:
# install openmodelica from its repository
! for deb in deb deb-src; do echo "$deb http://build.openmodelica.org/apt `lsb_release -cs` release"; done | sudo tee /etc/apt/sources.list.d/openmodelica.list
! wget -q http://build.openmodelica.org/apt/openmodelica.asc -O- | sudo apt-key add -
! apt update && apt install openmodelica

# check 'omc' command
! omc --help

In [ ]:
# install Funz through pypi
!pip install --upgrade https://github.com/Funz/Funz.py/tarball/master
!#pip install Funz
import Funz

In [ ]:
# install plugin to support Modelica I/O
Funz.installModel('Modelica', edit_script=False)

In [ ]:
# setup main input file
%%writefile NewtonCooling.mo
// @ref http://book.xogeny.com/behavior/equations/physical/
model NewtonCooling "An example of Newton's law of cooling"
  parameter Real T_inf=25 "Ambient temperature";
  parameter Real T0=90 "Initial temperature";
  parameter Real h=0.7 "Convective cooling coefficient";
  parameter Real A=1.0 "Surface area";
  parameter Real m=0.1 "Mass of thermal capacitance";
  parameter Real c_p=1.2 "Specific heat";
  Real T "Temperature";
initial equation
  T = T0 "Specify initial value for T";
equation
  m*c_p*der(T) = h*A*(T_inf-T) "Newton's law of cooling";
end NewtonCooling;

In [ ]:
# wake up the Funz 'daemon' which will provide calculation services:
Funz.startCalculators(1)

In [ ]:
# run just this file with Funz (that will check everything is ok for later parametric study)
Funz.Run(model="Modelica",input_files="NewtonCooling.mo")

In [ ]:
# setup main parametrized (with `$convection`) input file
%%writefile NewtonCooling.mo.par
// @ref http://book.xogeny.com/behavior/equations/physical/
model NewtonCooling "An example of Newton's law of cooling"
  parameter Real T_inf=25 "Ambient temperature";
  parameter Real T0=90 "Initial temperature";
  parameter Real h=$convection "Convective cooling coefficient";
  parameter Real A=1.0 "Surface area";
  parameter Real m=0.1 "Mass of thermal capacitance";
  parameter Real c_p=1.2 "Specific heat";
  Real T "Temperature";
initial equation
  T = T0 "Specify initial value for T";
equation
  m*c_p*der(T) = h*A*(T_inf-T) "Newton's law of cooling";
end NewtonCooling;

In [ ]:
# launch 6 calculations for different `convection` values (0.5, 0.6, 0.7, 0.8, 0.9, 1.0)
Funz.Run(model="Modelica",input_files="NewtonCooling.mo.par", input_variables={'convection':[0.5,0.6,0.7,0.8,0.9,1.0]}, output_expressions="min(T)")

In [ ]:
# install Brent root finding algorithm
Funz.installDesign('Brent')

In [ ]:
# find the `convection` value leading to `min(T) = 25.2` (with relative precision of 0.01 on `convection` value)
Funz.RunDesign(model="Modelica",input_files="NewtonCooling.mo.par", input_variables={'convection':"[0.5,1.0]"}, output_expressions="min(T)", 
               design="Brent", design_options={'ytarget':25.2, 'ytol':0.01})